In [1]:
import numpy as np
import pandas as pd
import gc
import sys
import os
import yaml
import re
import random
import math
import matplotlib.pyplot as plt
import warnings
import pickle
from typing import *
from pathlib import Path
from dataclasses import dataclass, field, asdict
from shutil import copyfile
warnings.simplefilter('ignore')

In [2]:
@dataclass
class Config:
    outdir: str = "../results/efficientnet"
    device: str = "cuda:1"
    device_id: int = 1
        
    tf_expt: int = -1 # tf.data.experimental.AUTOTUNE

    datadir: str = '../data/tfrecord-skf'
#     modeldir: str = '../models/bert/bert_en_uncased_L-24_H-1024_A-16_1'
    seed: int = 123
    valid_ratio: float = 0.25
    image_size: List[int] = field(default_factory=lambda: [512, 512])
    
    # Training config
    en_type: str = 'B0'
    batch_size: int = 32
    epochs: int = 20
    lr: float = 0.001

    def update(self, param_dict: Dict) -> "Config":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self
    
    def to_yaml(self, filepath: str, width: int = 120):
        with open(filepath, 'w') as f:
            yaml.dump(asdict(self), f, width=width)

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [4]:
base_dir = Path().resolve()
sys.path.append(os.path.abspath(base_dir / '../'))

config_dict = {
#     'epochs': 1,
}
config = Config().update(config_dict)
config.to_yaml(base_dir / config.outdir / 'config.yaml')

os.environ["CUDA_VISIBLE_DEVICES"] = str(config.device_id)

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn

from src.tokenization import *
from src.preprocess import *
from src.image import *
from src.model import *

In [7]:
# Data augmentation function
def data_augment(posting_id, image, label_group, matches):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)
    return posting_id, image, label_group, matches

In [9]:
def get_lr_callback(batch_size):
    lr_start   = 0.000001
    lr_max     = 0.000005 * 256
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

In [10]:
def count_data_items(filenames):
    # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [11]:
tfrecord_files = sorted(tf.io.gfile.glob(str(base_dir / config.datadir) + '/*.tfrec'))
# train_files, valid_files = tfrecord_files[:int(len(tfrecord_files) * (1 - config.valid_ratio))], tfrecord_files[int(len(tfrecord_files) * (1 - config.valid_ratio)):]
train_files, valid_files = tfrecord_files[int(len(tfrecord_files) * config.valid_ratio):], tfrecord_files[:int(len(tfrecord_files) * config.valid_ratio)]
steps_per_epoch = count_data_items(train_files) // config.batch_size

In [12]:
train = pd.read_csv(base_dir / config.datadir / 'train_folds.csv')
n_classes = train['label_group'].nunique()
n_classes

11014

In [13]:
seed_everything(config.seed)

outdir = base_dir / config.outdir / config.en_type

train_dataset = get_training_dataset(train_files, config, data_augment)
train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))

valid_dataset = get_validation_dataset(valid_files, config, data_augment)
valid_dataset = valid_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))

model = build_efficientnet_model(
    n_classes=n_classes,
    image_size=config.image_size,
    lr=config.lr,
    en_type=config.en_type,
    train=True
)

# Model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    str(outdir / (f'EfficientNet{config.en_type}_{config.seed}_' + '{epoch:02d}.h5')),
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True,
    save_weights_only = True, 
    mode = 'min'
)

history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs = config.epochs,
    callbacks = [checkpoint, get_lr_callback(config.batch_size)], 
    validation_data = valid_dataset,
    verbose = 1
)

pickle.dump(history.history, open(str(outdir / 'history.pkl'), 'wb'))

tf.keras.backend.clear_session()


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-06.
Epoch 1/20
856/856 [==============================] - ETA: 0s - loss: 23.9871 - sparse_categorical_accuracy: 0.0000e+00
Epoch 00001: val_loss improved from inf to 24.06334, saving model to /home/yamaguchi-milkcocholate/Shopee/notebooks/../results/efficientnet/B0/EfficientNetB0_123_01.h5
856/856 [==============================] - 675s 789ms/step - loss: 23.9871 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 24.0633 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: LearningRateScheduler reducing learning rate to 0.00025680000000000006.
Epoch 2/20
856/856 [==============================] - ETA: 0s - loss: 19.5395 - sparse_categorical_accuracy: 0.0078
Epoch 00002: val_loss improved from 24.06334 to 18.93654, saving model to /home/yamaguchi-milkcocholate/Shopee/notebooks/../results/efficientnet/B0/EfficientNetB0_123_02.h5
856/856 [==============================] - 655s 765ms/step - loss: 19.5395 - sparse_c